In [1]:
#Import relevant libraries
import astropy
import numpy as np
import ccdproc
import os
import glob
import matplotlib.pyplot as plt
from astropy.nddata import CCDData
import scipy

In [2]:
#TASK 1
#Here we are subtracing the master bias and master dark from the first half of master science B and dividing by the masterflat


masterflatB = astropy.nddata.CCDData.read('C:/Users/steve/OneDrive - University of Birmingham/LH Observatory Laboratory/Glob Cluter Final Code/GlobClusters/vexpflatB.fits', unit='adu')
masterbias = astropy.nddata.CCDData.read('C:/Users/steve/OneDrive - University of Birmingham/LH Observatory Laboratory/Glob Cluter Final Code/GlobClusters/expbias.fits', unit='adu')
masterdark = astropy.nddata.CCDData.read('C:/Users/steve/OneDrive - University of Birmingham/LH Observatory Laboratory/Glob Cluter Final Code/GlobClusters/expdark.fits', unit='adu')


#file_location = os.path.join('20191111/', 'science_B/', '*.fits')
#file_location = os.path.join('GlobularClusters/', 'science_B/', '*.fits')
filenames = glob.glob('C:/Users/steve/OneDrive - University of Birmingham/LH Observatory Laboratory/Glob Cluter Final Code/GlobClusters/20191111/science_B/*.fits')

scienceimagesB = []

for file in filenames:
    scienceImg = astropy.nddata.CCDData.read(file, unit='adu')
    hdul = astropy.nddata.CCDData.to_hdu(scienceImg)
    scienceSub1 = scienceImg.subtract(masterbias)
    scienceSub2 = scienceSub1.subtract(masterdark)
    divscience = scienceSub2.divide(masterflatB)
    #divscience2 = divscience.divide(60) #to ensure exposure time is 1 sec however we did not do this, we get kept exposure time at 60 seconds
    scienceimagesB.append(divscience)
    
    
print(scienceimagesB)

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


[CCDData([[ 351.37553037,  399.81337575,  399.14734404, ...,  350.00052568,
           418.32771038,   13.62226875],
         [ 416.61852726,  392.10962054,  370.61808174, ...,  381.23658153,
           370.05117617, -129.89498587],
         [ 371.49455226,  386.73037158,  385.00656274, ...,  366.16317553,
           404.31811011,  131.01469825],
         ...,
         [ 353.81264872,  360.53783935,  377.21175736, ...,  343.68529085,
           363.32370035,  131.24897187],
         [ 385.01040305,  361.8493286 ,  399.20207307, ...,  381.12483197,
           414.70444292, -432.38655888],
         [ 359.18627691,  367.75247066,  396.77139719, ...,  363.54602409,
           359.44084535, -264.52699092]], unit=''), CCDData([[ 306.90655003,  340.83037014,  317.33674755, ...,  331.30571849,
           364.50777104,   -5.4489075 ],
         [ 352.04661921,  357.2649136 ,  311.16371477, ...,  369.16349092,
           364.21484519, -110.9519671 ],
         [ 283.92335724,  368.72962285,  299.2

In [3]:
from skimage import data, draw
from skimage.registration import phase_cross_correlation
from scipy import ndimage


#Here we corrected for the phase shift for the first half of master science B. By aligning all the frames, we create a much more cleaned and precise image
scienceimagesBcorr = []
for i in range(len(scienceimagesB)):
    detected_shiftB = phase_cross_correlation(np.array(scienceimagesB[0]), np.array(scienceimagesB[i]), upsample_factor = 1000)
    print(detected_shiftB[0])
    scienceimagesB1 = scipy.ndimage.shift(scienceimagesB[i], detected_shiftB[0])
    scienceimagesB11 = astropy.nddata.CCDData(scienceimagesB1, unit = 'adu')
    scienceimagesBcorr.append(scienceimagesB11)

[0. 0.]
[-0.117 -0.043]
[-0.083 -0.133]
[-0.22   0.056]
[-0.251 -0.087]
[-0.25   0.028]
[-0.256  0.058]
[-0.26  -0.037]
[-2.438  0.539]
[-2.803  0.   ]
[-2.524  0.411]


In [4]:
scienceimagesBcorr = astropy.nddata.CCDData(scienceimagesBcorr, unit='adu')
print(scienceimagesBcorr)

[[[ 351.37553037  399.81337575  399.14734404 ...  350.00052568
    418.32771038   13.62226875]
  [ 416.61852726  392.10962054  370.61808174 ...  381.23658153
    370.05117617 -129.89498587]
  [ 371.49455226  386.73037158  385.00656274 ...  366.16317553
    404.31811011  131.01469825]
  ...
  [ 353.81264872  360.53783935  377.21175736 ...  343.68529085
    363.32370035  131.24897187]
  [ 385.01040305  361.8493286   399.20207307 ...  381.12483197
    414.70444292 -432.38655888]
  [ 359.18627691  367.75247066  396.77139719 ...  363.54602409
    359.44084535 -264.52699092]]

 [[ 308.92743071  341.11527218  317.81783545 ...  337.32529959
    351.53455541    0.        ]
  [ 348.74002092  359.52618499  309.10543048 ...  374.23291169
    344.56968784    0.        ]
  [ 283.78235724  364.31978613  298.30079443 ...  319.01073338
    328.02739964    0.        ]
  ...
  [ 329.50623335  345.47658601  311.20271413 ...  352.24757369
    288.21848399    0.        ]
  [ 358.05750889  323.52514153  334.

In [5]:
print((scienceimagesBcorr.shape))

(11, 4096, 4096)


In [6]:
#Here we combine all the corrected images to complete the first half of our master bias 
ScienceB = ccdproc.combine((scienceimagesBcorr), method = 'median', format = 'fits')
hdu1 = CCDData.to_hdu(ScienceB)
hdu1.writeto('MasterScienceB1.fits', overwrite = True)